In [1]:
import os
import matplotlib.pyplot as plt
import scipy.io as sio
import torch
import numpy as np
import pandas as pd
import logging
import re

In [2]:
WD = '/local/meliao/projects/fourier_neural_operator/experiments/10_linear_approx/'
os.chdir(WD)
print(os.getcwd())

/local/meliao/projects/fourier_neural_operator/experiments/10_linear_approx


In [3]:
from train_models import SpectralConv1d, FNO1dComplexTime, TimeDataSetLinearResiduals

In [4]:
# TRAIN_DF = '/local/meliao/projects/fourier_neural_operator/experiments/10_linear_approx/results/00_residual_train.txt'
# TEST_DF = '/local/meliao/projects/fourier_neural_operator/experiments/10_linear_approx/results/00_residual_test.txt'
DATA_FP = '/local/meliao/projects/fourier_neural_operator/data/2021-06-24_NLS_data_04_test.mat'
MODEL_FP = '/local/meliao/projects/fourier_neural_operator/experiments/10_linear_approx/models/00_residual_ep_500'
BASELINE_FP = '/local/meliao/projects/fourier_neural_operator/experiments/08_FNO_pretraining/models/00_pretrain_ep_1000'
# PLOTS_DIR = '/local/meliao/projects/fourier_neural_operator/experiments/09_predict_residuals/plots/'
PLOTS_DIR = 'plots/'
RESULTS_DIR = 'results/'


In [5]:
def make_train_test_plot(a_train, a_test, fp=None, title=""):
    fig, ax = plt.subplots(1, 2, sharey=False)
    fig.patch.set_facecolor('white')
#     fig.set_size_inches(6.4, 5.2)


    # a_train and a_test are the time-dependent FNO data. They're in the first column
    ax[0].set_title("Train")
    ax[0].plot(a_train.epoch, a_train.MSE, '-', color='red', label='train')
    ax[1].plot(a_test.epoch, a_test.test_mse, '--', color='red', label='test')
    ax[0].set_xlabel("Epoch", fontsize=13)
    ax[1].set_xlabel("Epoch", fontsize=13)
    # ax[0].legend()
    ax[0].set_yscale('log')

    ax[0].set_ylabel("MSE", fontsize=13)

    # b_train and b_test are the time-dependent. They're in the seecond column
    ax[1].set_title("Test")
    # ax[1].plot(b_train.epoch, b_train.MSE, '-', color='blue', label='train')
    # ax[1].plot(b_test.epoch, b_test.test_mse, '--', color='blue', label='test')
    ax[1].set_xlabel("Epoch", fontsize=13)
    ax[1].set_yscale('log')
    # ax[1].legend(fontsize=13)

    fig.suptitle(title)

#     plt.tight_layout()
    fig.tight_layout(rect=[0, 0.03, 1, 0.95])


    if fp is not None:
        plt.savefig(fp)
    else:
        plt.show()
    plt.close(fig)

In [6]:
TEST_DF = 'results/00_one_step_test.txt'

In [8]:
test_df = pd.read_table(TEST_DF)
test_df = test_df.sort_values('test_mse')
print(test_df)

epoch  test_l2_normalized_error  test_mse
19    950                  0.063078  0.001896
18    900                  0.063134  0.001900
17    850                  0.063243  0.001906
16    800                  0.063350  0.001913
15    750                  0.063556  0.001925
14    700                  0.063760  0.001938
13    650                  0.064160  0.001962
12    600                  0.064557  0.001987
11    550                  0.065341  0.002035
10    500                  0.066133  0.002085
9     450                  0.067740  0.002188
8     400                  0.069419  0.002298
7     350                  0.073001  0.002541
6     300                  0.077038  0.002828
5     250                  0.087127  0.003615
4     200                  0.100252  0.004770
3     150                  0.133292  0.008305
2     100                  0.175168  0.014055
1      50                  0.243926  0.027100
0       0                  0.723287  0.251400


In [9]:
MODEL_FP = 'models/00_one_step_ep_1000'
FNO_T_MODEL = '/local/meliao/projects/fourier_neural_operator/experiments/07_long_time_dependent_runs/models/00_time_dep_ep_1000'
DATA_FP = '/local/meliao/projects/fourier_neural_operator/data/2021-06-24_NLS_data_04_test.mat'
# BASELINE_FP = '/local/meliao/projects/fourier_neural_operator/experiments/08_FNO_pretraining/models/00_pretrain_ep_1000'

# DATA_FP = '/local/meliao/projects/fourier_neural_operator/data/2021-06-24_NLS_data_04_test.mat'
# MODEL_FP = '/local/meliao/projects/fourier_neural_operator/experiments/09_predict_residuals/models/00_residual_ep_500'
BASELINE_FP = '/local/meliao/projects/fourier_neural_operator/experiments/08_FNO_pretraining/models/00_pretrain_ep_1000'


In [10]:
d = sio.loadmat(DATA_FP)
tdlrp_model = torch.load(MODEL_FP, map_location='cpu')
fno_model = torch.load(BASELINE_FP, map_location='cpu')
fno_t_model = torch.load(FNO_T_MODEL, map_location='cpu')

In [11]:
t_dset = TimeDataSetLinearResiduals(d['output'], d['t'], d['x'])
# t_dloader = torch.utils.data.DataLoader(t_dset, batch_size=1, shuffle=False)

In [12]:
def load_data(fp):
    logging.info("Loading data from {}".format(fp))
    data = sio.loadmat(os.path.expanduser(fp))
    return data['output'], data['t']

def load_model(fp, device):
    # Model datatypes are loaded from train_models.py
    model = torch.load(fp, map_location=device)
    return model

def l2_normalized_error(pred, actual):
    """Short summary.

    Parameters
    ----------
    pred : type
        Description of parameter `pred`.
    actual : type
        Description of parameter `actual`.

    Returns
    -------
    types
        Description of returned object.

    """
    errors = pred - actual
    error_norms = torch.linalg.norm(torch.tensor(errors), dim=-1, ord=2)
    actual_norms = torch.linalg.norm(torch.tensor(actual), dim=-1, ord=2)
    normalized_errors = torch.divide(error_norms, actual_norms)
    return normalized_errors.detach().numpy()
def prepare_input(X):
    # X has shape (nbatch, 1, grid_size)
    s = X.shape[-1]
    n_batches = X.shape[0]

    # Convert to tensor
    X_input = torch.view_as_real(torch.tensor(X, dtype=torch.cfloat))

    # FNO code appends the spatial grid to the input as below:
    x_grid = torch.linspace(-np.pi, np.pi, 1024).view(-1,1)
    X_input = torch.cat((X_input, x_grid.repeat(n_batches, 1, 1)), axis=2)

    return X_input

In [13]:
def test_TDLRP_predictions(model, fno_model, fno_t_model, t_dset):

    # X has shape (nbatch, n_tsteps, grid_size)
    TEST_KEYS = ['TDLRP', 'FNO', 'TD-FNO']
    preds_dd = {i: np.zeros((t_dset.X.shape[0], t_dset.X.shape[1]-1, t_dset.X.shape[2]), dtype=np.cdouble) for i in TEST_KEYS}
    errors_dd = {i: np.zeros((t_dset.X.shape[0], t_dset.X.shape[1]-1), dtype=np.double) for i in TEST_KEYS}

    one_tensor = torch.tensor(1, dtype=torch.float).repeat([t_dset.X.shape[0],1,1])
    IC_input = prepare_input(t_dset.X[:,0,:])

    # First input is given by the initial condition
    comp_input_i = prepare_input(t_dset.X[:,0,:])
    comp_input_j = prepare_input(t_dset.X[:,0,:])
    # Iterate along timesteps
    for i in range(t_dset.t.shape[0]-1):
        SOLN_I = torch.tensor(t_dset.X[:,i+1,:])
        # First test: composing the model
        comp_preds_i = fno_model(comp_input_i, one_tensor).detach().numpy()
        # comp_preds_i = emulator(comp_input_i).detach().numpy()
        preds_dd['FNO'][:,i,:] = comp_preds_i
        comp_input_i = prepare_input(comp_preds_i)
        errors_i = l2_normalized_error(torch.tensor(comp_preds_i), SOLN_I)
        errors_dd['FNO'][:,i] = errors_i

        # Second test: prediction from the TDLRP
        # if time_dep_model:
        i_tensor = torch.tensor(t_dset.t[i+1], dtype=torch.float).repeat([t_dset.X.shape[0],1,1])
        preds_k = model(comp_input_j, one_tensor).detach().numpy() + t_dset.linear_part_arr[:,i+1].numpy()
        preds_dd['TDLRP'][:,i,:] = preds_k
        comp_input_j = prepare_input(preds_k)
        errors_k = l2_normalized_error(torch.tensor(preds_k), SOLN_I)
        errors_dd['TDLRP'][:,i] = errors_k

        # Third test: prediction from FNO-T model
        preds_j = fno_t_model(IC_input, i_tensor).detach().numpy()
        preds_dd['TD-FNO'][:,i] = preds_j
        errors_dd['TD-FNO'][:,i] = l2_normalized_error(torch.tensor(preds_j), SOLN_I)
    # for k,v in errors_dd.items():
    #     print("{}: {}: {}".format(k, v.shape, v))
    return preds_dd, errors_dd



In [14]:
preds_dd, errors_dd = test_TDLRP_predictions(tdlrp_model, fno_model, fno_t_model, t_dset)

In [15]:
def plot_time_errors(errors_dd, t_grid, title, fp):
    # SHOW_N_TSTEPS = t_grid.shape[1]
    SHOW_N_TSTEPS = 4

    n_t_steps = t_grid.shape[1]
    x_vals = t_grid.flatten()[:SHOW_N_TSTEPS]
    plt.figure().patch.set_facecolor('white')
    for k, v in errors_dd.items():
        print("{}: {}".format(k, v.shape))
        v_means = np.mean(v, axis=0)[:SHOW_N_TSTEPS]
        v_stds = np.std(v, axis=0)[:SHOW_N_TSTEPS]
        plt.plot(x_vals, v_means, label=k, alpha=0.7)
        plt.fill_between(x_vals,
                            v_means + v_stds,
                            v_means - v_stds,
                            alpha=0.3)
    plt.legend()
    plt.xlabel("Time step")
    plt.xticks(ticks=np.arange(0, SHOW_N_TSTEPS),
               labels=make_special_ticks(SHOW_N_TSTEPS),
              rotation=45,
              ha='right',
              )
    plt.ylabel("$L_2$-Normalized Errors")
    # plt.yscale('log')
    plt.title(title)
    plt.tight_layout()
    plt.savefig(fp)
    plt.clf()


def make_special_ticks(n):
    s = "$t={} \\ \\to  \\ t={}$"
    return [s.format(i, i+1) for i in range(n)]


In [16]:
fp_time_errors = os.path.join(PLOTS_DIR, 'one_step_time_errors.png')
errors_dd_i = {k:np.delete(v, [59], axis=0) for k,v in errors_dd.items()}
plot_time_errors(errors_dd_i, d['t'][:,:-1], "FNO vs. TDLRP: Test Dataset", fp_time_errors)

TDLRP: (99, 20)
FNO: (99, 20)
TD-FNO: (99, 20)


<Figure size 432x288 with 0 Axes>

In [40]:
def plot_one_testcase_heatmap_T(preds, resids, model_name, fp=None):
    INTERPOLATION = None
    CMAP='hot'
    CMAP_2='hot'
    XT, XT_STR = gen_ticks(0, 1023, -np.pi, np.pi, 5)
    ASPECT= (preds.shape[1] / preds.shape[0]) * 100/256

    # preds has shape (n_tsteps, grid_size)
    errors = np.abs(preds - resids)
    fig, ax = plt.subplots(3, 1, sharex=True, sharey=True)
    fig.set_size_inches(12.8, 9.6)
    im_0 = ax[0].imshow(np.abs(resids), interpolation=INTERPOLATION, aspect=ASPECT, cmap=CMAP)
    ax[0].set_title("NLS Nonlinear Residuals: $|u|$", fontsize=20)
    ax[2].set_xticks(XT)
    ax[2].set_xticklabels(XT_STR)
    ax[0].set_ylabel("$t$", fontsize=20)
    ax[1].set_ylabel("$t$", fontsize=20)
    im_1 = ax[1].imshow(np.abs(preds), interpolation=INTERPOLATION, aspect=ASPECT, cmap=CMAP)
    ax[1].set_title("{} Predictions: $|\\hat u |$".format(model_name), fontsize=20)
    im_2 = ax[2].imshow(errors, interpolation=INTERPOLATION, aspect=ASPECT, cmap=CMAP_2)
    ax[2].set_title("Absolute Errors: $| \\hat u - u |$", fontsize=20)
    ax[2].set_ylabel("$t$", fontsize=20)
    ax[2].set_xlabel("$x$", fontsize=20)
    fig.colorbar(im_0, ax=ax[0])
    fig.colorbar(im_1, ax=ax[1])
    fig.colorbar(im_2, ax=ax[2])
    if fp is not None:
        plt.savefig(fp)
    else:
        plt.show()
    plt.close(fig)


def gen_ticks(lb_im, ub_im, lb_world, ub_world, n_ticks):
    tick_locs = np.linspace(lb_im, ub_im, n_ticks)
    dist_world = ub_world - lb_world
    tick_names_flt = [lb_world + dist_world * i /ub_im for i in tick_locs]
    tick_names_str = ["{:.2f}".format(i) for i in tick_names_flt]
    return tick_locs, tick_names_str

def plot_one_testcase_panels(preds, residuals, fp=None):
    SHOW_T_TIMESTEPS = 20
    ALPHA = 0.7 # Controls saturation of overlapping lines. 1 is full saturation, 0 is none.
    # preds has shape (n_tsteps, grid_size)
    _, grid_size = preds.shape
    n_tsteps = SHOW_T_TIMESTEPS

    fig, ax = plt.subplots(n_tsteps-1, 3, sharex='col', sharey=False)
    fig.set_size_inches(15,20) #15, 20 works well
    ax[0,0].set_title("$Re(u)$", size=20)
    ax[0,1].set_title("$Im(u)$", size=20)
    ax[0,2].set_title("$| u - \\hat u|$", size=20)

    # residuals = solns - lin_part
    # preds = preds - lin_part
    # preds = TDLRP + lin_part
    # errors = solns - preds = solns - TDLRP - lin_part
    errors = residuals - preds

    for i in range(1, n_tsteps):
        # First column has Re(prediction), Re(solution)
        ax[i-1,0].plot(np.real(preds[i]), alpha=ALPHA, label='TDLRP preds')
        ax[i-1,0].plot(np.real(residuals[i]), alpha=ALPHA, label='residuals')
        ax[i-1,0].set_ylabel("t = {}".format(i), size=15)

        # Second column has Im(prediction), Im(solution)
        ax[i-1,1].plot(np.imag(preds[i]), alpha=ALPHA, label='TDLRP preds')
        ax[i-1,1].plot(np.imag(residuals[i]), alpha=ALPHA, label='residuals')

        # Third column is Abs(prediction - solution)
        ax[i-1,2].plot(np.real(errors[i]), label='Real errors', color='red')
        ax[i-1,2].plot(np.imag(errors[i]), label='Imag errors', color='green')
        # ax[i,2].plot(b_errors_i, label='FNO baseline errors', color='green')
        ax[i-1,2].hlines(0, xmin=0, xmax=errors.shape[1]-1, linestyles='dashed')

    ax[0,0].legend(fontsize=13, markerscale=2)
    ax[0,1].legend(fontsize=13, markerscale=2)
    ax[0,2].legend(fontsize=13, markerscale=2)
    if fp is not None:
        plt.savefig(fp)
    else:
        plt.show()
    plt.close(fig)



In [44]:
for i in range(1):
    preds_i = np.zeros(t_dset.X[i].shape, dtype=np.cfloat) #t_dset.X[i].numpy()
    preds_i[1:] = preds_dd['TDLRP'][i]
    preds_i = preds_i - t_dset.linear_part_arr[i].numpy()
    solns_i = t_dset.X[i].numpy() 
    lin_part_i = t_dset.linear_part_arr[i].numpy()
    resids_i = solns_i - lin_part_i
    model_name = 'TDLRP'
    fp_i = os.path.join(PLOTS_DIR, 'heatmap_test_case_{}.png'.format(i))
    plot_one_testcase_heatmap_T(preds_i, resids_i, model_name) #, fp=fp_i)
    # plot_one_testcase_heatmap_T(preds_i + lin_part_i, solns_i, model_name) #, fp=fp_i)
    plot_one_testcase_panels(preds_i, resids_i)

ValueError: could not broadcast input array from shape (20,1024) into shape (21,1023)